In [1]:
import numpy as np
import numpy.ma as ma
import h5py
import sys
import argparse
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/a/antonio-costa/BehaviorModel/utils/')
import delay_embedding as embed
import operator_calculations as op_calc
import clustering_methods as cl
import stats

In [2]:
condition_labels = ['Light (5x5cm)','Light (1x5cm)','Looming(5x5cm)','ChasingDot coarsespeeds(5x5cm)','ChasingDot finespeeds(5x5cm)','Dark_Transitions(5x5cm)',
                    'Phototaxis','Optomotor Response (1x5cm)','Optokinetic Response (5x5cm)','Dark (5x5cm)','3 min Light<->Dark(5x5cm)',
                    'Prey Capture Param. (2.5x2.5cm)','Prey Capture Param. RW. (2.5x2.5cm)',
                    'Prey Capture Rot.(2.5x2.5cm)','Prey capture Rot. RW. (2.5x2.5cm)','Light RW. (2.5x2.5cm)']
condition_recs = np.array([[515,525],[160,172],[87,148],[43,60],[22,43],[60,87],
                           [202,232],[148,160],[172,202],[505,515],[0,22],
                           [232,301],[347,445],[301,316],[316,347],
                           [445,505]])

In [3]:
f= h5py.File('/bucket/StephensU/antonio/zebrabouts/pool_ex7_PCs.h5','r')
print(f.keys())
print(f['MetaData'].keys())
cov = np.array(f['cov'])
data_means = np.array([f['data_means']])
eigvecs = np.array(f['eigvecs'])
max_shuffles = np.array(f['max_shuffs'])
pca_fish = ma.masked_invalid(np.array(f['pca_fish']))
var_exp = np.array(f['var_exp'])
lenghts_data = np.array([f['MetaData/lengths_data']])
f.close()
pca_fish[pca_fish==0]=ma.masked

<KeysViewHDF5 ['MetaData', 'cov', 'data_means', 'eigvecs', 'max_shuffs', 'pca_fish', 'seeds', 'var_exp']>
<KeysViewHDF5 ['lengths_data']>


In [5]:
c0,cf  = condition_recs[4]
print(condition_labels[4])
#5s stimuli every 2mins
X_sample = pca_fish[c0:cf]
nbouts = np.array([ma.count(X_sample[kf,:,0]) for kf in range(len(X_sample))])
sel = nbouts>7500
X_sample = X_sample[sel]

In [17]:
X_sample.shape

(11, 15066, 50)

In [15]:
f = h5py.File('/flash/StephensU/antonio/zebrafish_data.h5','w')
cov_ = f.create_dataset('cov',cov.shape)
cov_[...]=cov
dmeans_ = f.create_dataset('data_means',data_means.shape)
dmeans_[...]=data_means
eigvecs_ = f.create_dataset('eigvecs',eigvecs.shape)
eigvecs_[...] = eigvecs
max_shuffs_ = f.create_dataset('max_shuffs',max_shuffles.shape)
max_shuffs_[...] = max_shuffles
varexp_ = f.create_dataset('var_exp',var_exp.shape)
varexp_[...] = var_exp
pca_fish_ = f.create_dataset('pca_fish',X_sample.shape)
pca_fish_[...] = X_sample
f.close()